In [1]:
import popart
import onnx
from onnx.helper import make_graph, make_attribute, make_node
from onnx import TensorProto, SequenceProto
import numpy as np
from typing import *
import time


In [ ]:
from onnx import 

In [2]:
m = onnx.load("qtc211-tf-ipu/qtcv211-wo-kv.onnx")

In [7]:
for inp in m.graph.input:
    inp.type.tensor_type.elem_type = 6

In [ ]:
m.ir_version

In [ ]:
onnx.checker.check_model(m, 1)

In [ ]:
m.graph.node

In [ ]:
m.graph.node[2].attribute[0].i = 6

In [ ]:
onnx.save(m, "ms.onnx")

In [ ]:
transformed_mode = popart.GraphTransformer("model.onnx")
transformed_mode.convertINT64ToINT32()

In [ ]:
tm = onnx.load_from_string(transformed_mode.getModelProto())

In [ ]:
tm.graph.input

In [ ]:
tm.graph.node

In [ ]:
tm.graph.node.insert(0, make_node("Cast", ["const_fold_opt__8"], ["const_fold_opt_int64_8"], to = 7))

In [ ]:
tm.graph.node[0]

In [ ]:
tm.graph.node[2]

In [ ]:
tm.graph.node[2].input[1] = "const_fold_opt_int64_8"

In [ ]:
tm.graph.node[2].attribute[0].i = 6

In [ ]:
onnx.checker.check_model(tm, 1)

In [ ]:
onnx.save(tm, "model_conv.onnx")

In [ ]:
tm.graph.node[3].attribute[0].i = 6

In [ ]:
m.graph.node[1].attribute[0].i = 6

In [ ]:
[n for n in m.graph.node if n.op_type == "Cast" and n.attribute[0].i == 7]

In [ ]:
len(m.graph.node)

In [ ]:
def make_an_anchor(onnx_model_builder: popart.Builder, anchor_return_type: str = "ALL"):
    return {output_name: popart.AnchorReturnType(anchor_return_type) 
                for output_name in onnx_model_builder.getOutputTensorIds()}

def set_batch_size(shape: List, batch_size = 1):
    """ shape is like [unknow_batch_size, n1, n2, ...] """
    shape[0] = batch_size if shape[0] == 0 else shape[0]
    return shape

def convert_popart_dtype(dtype: str):
    dtype_conv_dic = {
        "int64": "INT32",
        "int32": "INT32",
        "float32": "FLOAT",
        "float16": "FLOAT16",
        "float64": "FLOAT",
    }

    return dtype_conv_dic[dtype]

def convert_numpy_dtype(dtype: str):
    dtype_conv_dic = {
        "int64": np.int32, 
        "int32": np.int32, 
        "float32": np.float32,
        "float": np.float32,
        "float64": np.float32,
    }
    return dtype_conv_dic[dtype.lower()]


def add_shapeinfo_from_onnx(onnx_model_builder: popart.Builder, batch_size = 1, batch_per_step = 1):
    inputs_tensor_id = onnx_model_builder.getInputTensorIds()
    outputs_tensor_id = onnx_model_builder.getOutputTensorIds()

    print(inputs_tensor_id)
    inputs_shapes = [set_batch_size(onnx_model_builder.getTensorShape(i), 
                                batch_size=batch_size * batch_per_step) for i in inputs_tensor_id]
    print(inputs_shapes)
    inputs_dtypes = [convert_popart_dtype(onnx_model_builder.getTensorDtypeString(i)) for i in inputs_tensor_id]
    print(inputs_dtypes)

    inputs_tensor_shapes = [set_batch_size(onnx_model_builder.getTensorShape(i), 
                                batch_size=batch_size) for i in inputs_tensor_id + outputs_tensor_id]
    inputs_tensor_dtypes = [convert_popart_dtype(onnx_model_builder.getTensorDtypeString(i)) for i in inputs_tensor_id + outputs_tensor_id]

    inputShapeInfo = popart.InputShapeInfo()

    for tid, tshape, tdype in zip(inputs_tensor_id + outputs_tensor_id, inputs_tensor_shapes, inputs_tensor_dtypes):
        inputShapeInfo.add(tid, popart.TensorInfo(tdype, tshape))

    return inputs_tensor_id, inputShapeInfo, inputs_shapes, inputs_dtypes


def fake_dataset(inputs_tensor_id, inputs_shapes, inputs_dtypes, num_samples = 100):
    for _ in range(num_samples):
        yield { i: np.random.randint(12, size=s).astype(convert_numpy_dtype(d)) for i, s, d in zip(inputs_tensor_id, inputs_shapes, inputs_dtypes) }

In [ ]:
batch_size = 1
batch_per_step = 1
global_batch_size = batch_per_step * batch_size
epochs = 80
n_sample = epochs + 10 * 2

n_sample = 1

builder = popart.Builder("qtc211-tf-ipu/qtcv211_wo_kv.onnx")
anchors = make_an_anchor(builder)
inputs_tensor_id, inputShapeInfo, inputs_shapes, inputs_dtypes = add_shapeinfo_from_onnx(builder, batch_size=batch_size, batch_per_step = batch_per_step)
# anchors = {output_name: popart.AnchorReturnType("All") for output_name in builder.getOutputTensorIds() }
dataflow = popart.DataFlow(batch_per_step, anchors)
device = popart.DeviceManager().acquireAvailableDevice(2)
# device = popart.DeviceManager().createCpuDevice()

opts = popart.SessionOptions()
opts.virtualGraphMode = popart.VirtualGraphMode.Auto
# opts.enablePipelining = True
partials_type = "half"
opts.partialsTypeMatMuls = partials_type
opts.convolutionOptions = {'partialsType': partials_type}
# opts.groupHostSync = True

session = popart.InferenceSession(builder.getModelProto(), dataflow, device, 
                                inputShapeInfo=inputShapeInfo,
                                userOptions=opts)

In [ ]:
builder.getTensorDtypeString('TensorDict/StandardKvParser:0')

In [34]:
[ n for n in m.graph.node if "reload_query/sub/x:0" in n.input]

[input: "reload_query/sub/x:0"
 input: "reload_query/ones_like__725:0"
 output: "reload_query/ones_like:0"
 name: "reload_query/ones_like"
 op_type: "Expand",
 input: "reload_query/sub/x:0"
 input: "reload_query/ToInt64_1:0"
 output: "reload_query/sub:0"
 name: "reload_query/sub"
 op_type: "Sub",
 input: "reload/semantics_cnn/strided_slice_4:0"
 input: "reload_query/sub/x:0"
 output: "reload/semantics_cnn/Equal_1:0"
 name: "reload/semantics_cnn/Equal_1"
 op_type: "Equal",
 input: "reload_qt/semantics_cnn/strided_slice_4:0"
 input: "reload_query/sub/x:0"
 output: "reload_qt/semantics_cnn/Equal_1:0"
 name: "reload_qt/semantics_cnn/Equal_1"
 op_type: "Equal"]

In [ ]:
for init in m.graph.initializer:
    if init.name in {"const_starts__708", "const_ends__709", "const_axes__728", "const_ends__727"} and init.data_type == TensorProto.INT64:
        init.data_type = TensorProto.INT32

In [ ]:
[init for init in m.graph.initializer if init.data_type == 7 and "const" in init.name]

In [53]:
onnx.checker.check_model(m, 1)

In [ ]:
m.graph.initializer[5].name

In [10]:
def get_info_by_name(model, name: str):
    for ordn, node in enumerate(model.graph.node):
        if node.name == name:
            return ('node', ordn, node)
    for ordn, init in enumerate(model.graph.initializer):
        if node.name == name:
            return ('initializer', ordn, init)
    return "Not Found"

In [11]:
get_info_by_name(m, "reload_query/NotEqual")

('node',
 10,
 input: "reload_query/strided_slice:0"
 input: "reload_qt/NotEqual/y:0"
 output: "reload_query/NotEqual:0"
 name: "reload_query/NotEqual"
 op_type: "Equal")

In [ ]:
onnx.save(m, "qtc211-tf-ipu/qtcv211-wo-kv-int32.onnx")

In [ ]:
m.graph.node[4]

In [23]:
[(i, n) for i, n in enumerate(m.graph.node) if "reload_query/concat_int32:0" in n.input]

[(9,
  input: "reload_query/concat_int32:0"
  input: "reload_qt/NotEqual/y:0"
  output: "reload_query/Equal:0"
  name: "reload_query/Equal"
  op_type: "Equal"),
 (23,
  input: "reload_query/sub:0"
  input: "reload_query/concat_int32:0"
  output: "reload_query/mul_3:0"
  name: "reload_query/mul_3"
  op_type: "Mul")]

In [52]:
[(i, n) for i, n in enumerate(m.graph.node) if "reload_query/zeros_like" == n.name]

[(1777,
  input: "Mul__714:0"
  output: "reload_query/zeros_like:0"
  name: "reload_query/zeros_like"
  op_type: "Cast"
  attribute {
    name: "to"
    i: 6
    type: INT
  }
  domain: "")]

In [43]:
[(i, init) for i, init in enumerate(m.graph.initializer) if "reload_query/mul_2/y:0" == init.name]

[(25,
  data_type: 6
  name: "reload_query/mul_2/y:0"
  raw_data: "f\000\000\000\000\000\000\000")]

In [51]:
m.graph.node[1777].attribute[0].i = 6

In [19]:
m.graph.node[22].input[1] = "reload_query/concat_int32:0"

In [ ]:
m.graph.node[7]

In [42]:
m.graph.initializer[25].data_type = 6

In [28]:
m.graph.node.insert(9, make_node("Cast", inputs=["reload_query/concat:0"], outputs=["reload_query/concat_int32:0"], to = 6))

In [26]:
m.graph.node.remove(m.graph.node[6])

In [54]:
onnx.save(m, "qtc211-tf-ipu/qtcv211-int32-manual.onnx")